This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Best practices for the real world

## Getting the most out of your models

### Hyperparameter optimization

#### Using KerasTuner

In [17]:
!pip install keras-tuner -q

**A KerasTuner model-building function**

In [1]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    units = hp.Int(name="units", min_value=16, max_value=64, step=16)
    model = keras.Sequential([
        layers.Dense(units, activation="relu"),
        layers.Dense(10, activation="softmax")
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

**A KerasTuner `HyperModel`**

In [2]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, num_classes):
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int(name="units", min_value=16, max_value=64, step=16)
        model = keras.Sequential([
            layers.Dense(units, activation="relu"),
            layers.Dense(self.num_classes, activation="softmax")
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"])
        model.compile(
            optimizer=optimizer,
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy"])
        return model

hypermodel = SimpleMLP(num_classes=10)

In [3]:
tuner = kt.BayesianOptimization(
    build_model,
    objective="val_accuracy",
    max_trials=10,
    executions_per_trial=2,
    directory="mnist_kt_test",
    overwrite=True,
)

Metal device set to: Apple M2


2024-06-11 16:01:44.473672: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-06-11 16:01:44.473856: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [5]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255
x_train_full = x_train[:]
y_train_full = y_train[:]
num_val_samples = 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:]
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:]
callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5),
]
tuner.search(
    x_train, y_train,
    batch_size=128,
    epochs=10,
    validation_data=(x_val, y_val),
    callbacks=callbacks,
    verbose=2,
)

Trial 10 Complete [00h 01m 07s]
val_accuracy: 0.9722000062465668

Best val_accuracy So Far: 0.9727999866008759
Total elapsed time: 00h 13m 38s


**Querying the best hyperparameter configurations**

In [6]:
top_n = 4
best_hps = tuner.get_best_hyperparameters(top_n)

In [7]:
def get_best_epoch(hp):
    model = build_model(hp)
    callbacks=[
        keras.callbacks.EarlyStopping(
            monitor="val_loss", mode="min", patience=10)
    ]
    history = model.fit(
        x_train, y_train,
        validation_data=(x_val, y_val),
        epochs=100,
        batch_size=128,
        callbacks=callbacks)
    val_loss_per_epoch = history.history["val_loss"]
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}")
    return best_epoch

In [8]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    model.fit(
        x_train_full, y_train_full,
        batch_size=128, epochs=int(best_epoch * 1.2))
    return model

best_models = []
for hp in best_hps:
    model = get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/100


2024-06-11 16:15:23.215979: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


390/391 [============================>.] - ETA: 0s - loss: 0.4161 - accuracy: 0.8908

2024-06-11 16:15:26.364124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


391/391 [==============================] - 4s 9ms/step - loss: 0.4156 - accuracy: 0.8909 - val_loss: 0.2288 - val_accuracy: 0.9361
Epoch 2/100
391/391 [==============================] - 3s 8ms/step - loss: 0.2103 - accuracy: 0.9398 - val_loss: 0.1710 - val_accuracy: 0.9522
Epoch 3/100
391/391 [==============================] - 3s 8ms/step - loss: 0.1605 - accuracy: 0.9537 - val_loss: 0.1422 - val_accuracy: 0.9599
Epoch 4/100
391/391 [==============================] - 3s 7ms/step - loss: 0.1317 - accuracy: 0.9622 - val_loss: 0.1263 - val_accuracy: 0.9649
Epoch 5/100
391/391 [==============================] - 3s 9ms/step - loss: 0.1128 - accuracy: 0.9679 - val_loss: 0.1172 - val_accuracy: 0.9657
Epoch 6/100
391/391 [==============================] - 3s 7ms/step - loss: 0.0988 - accuracy: 0.9716 - val_loss: 0.1178 - val_accuracy: 0.9665
Epoch 7/100
391/391 [==============================] - 4s 9ms/step - loss: 0.0879 - accuracy: 0.9743 - val_loss: 0.1089 - val_accuracy: 0.9690
Epoch 8/100

NameError: name 'model' is not defined

In [ ]:
best_models = tuner.get_best_models(top_n)

#### The art of crafting the right search space

#### The future of hyperparameter tuning: automated machine learning

### Model ensembling

## Scaling-up model training

### Speeding up training on GPU with mixed precision

#### Understanding floating-point precision

In [ ]:
import tensorflow as tf
import numpy as np
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array)
tf_tensor.dtype

In [ ]:
np_array = np.zeros((2, 2))
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32")
tf_tensor.dtype

#### Mixed-precision training in practice

In [ ]:
from tensorflow import keras
keras.mixed_precision.set_global_policy("mixed_float16")

### Multi-GPU training

#### Getting your hands on two or more GPUs

#### Single-host, multi-device synchronous training

### TPU training

#### Using a TPU via Google Colab

#### Leveraging step fusing to improve TPU utilization

## Summary